In [2]:
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.pipeline import *
from sklearn.compose import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
hr = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Cases/Cases/human-resources-analytics/HR_comma_sep.csv")
hr.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [4]:
# understand here target column is Left

In [5]:
X, y = hr.drop('left', axis =1), hr['left']
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=25)

In [6]:
# now doing onehotencoding using sklearn not pandas

In [7]:
ohe = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')
ct = make_column_transformer((ohe, make_column_selector(dtype_include=object)), 
                             ("passthrough", make_column_selector(dtype_exclude=object)),
                             verbose_feature_names_out=False)
ct = ct.set_output(transform='pandas')

In [8]:
lr = LogisticRegression(solver='lbfgs', penalty='l2')
pipe = Pipeline([('TRANSF',ct), ('LR',lr)])
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7992887308290731


/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [9]:
penalties = ['l1', 'l2', 'elasticnet', None]
ratios = np.linspace(0.001, 0.98, 20)
scores = []
for p in penalties:
    for r in ratios:
        lr = LogisticRegression(solver='saga', penalty=p, l1_ratio=r)
        pipe = Pipeline([('TRANSF',ct), ('LR',lr)])
        pipe.fit(X_train, y_train)
        y_pred = pipe.predict(X_test)
        scores.append([p,r,(accuracy_score(y_test, y_pred))])
df_scores = pd.DataFrame(scores, columns=['penality','l1_ratio', 'score'])
df_scores.sort_values('score', ascending=False).head(1)

/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings.warn(
/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:1196: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  warnings

,penality,l1_ratio,score
11,l1,0.567789,0.754168


In [10]:
# Inferencing

In [11]:
tst = pd.read_csv(r"/home/sarthakredasani/Documents/CDAC_ML/Cases/Cases/human-resources-analytics/tst_hr.csv")
tst.head(5)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,Department,salary
0,0.11,0.88,7,272,4,0,0,sales,medium
1,0.72,0.87,5,223,5,0,0,sales,low
2,0.37,0.52,2,159,3,0,0,sales,low
3,0.41,0.50,2,153,3,0,0,sales,low
4,0.67,0.61,3,202,2,0,0,technical,medium


In [12]:
lr = LogisticRegression(solver='lbfgs', penalty='l2')
pipe = Pipeline([('TRANSF',ct), ('LR',lr)])
pipe.fit(X, y)
pipe.predict(tst)

/home/sarthakredasani/.local/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


array([1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])